In [1]:
import mysql.connector
import pandas as pd
from kafka import KafkaProducer
from datetime import datetime
import time
import random
import numpy as np

In [2]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "root",
    database = "client_rw"
)

In [3]:
cursor = mydb.cursor()

In [4]:
query = "select * from transaction_summary"
cursor.execute(query)

In [5]:
result = cursor.fetchall()

In [6]:
columns = [col[0] for col in cursor.description]

In [7]:
df = pd.DataFrame(result, columns=columns)

In [8]:
df = df[["Account_No","WITHDRAWL_AMOUNT","DEPOSIT_AMT","BALANCE_AMT"]]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116201 entries, 0 to 116200
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Account_No        116201 non-null  object
 1   WITHDRAWL_AMOUNT  116201 non-null  object
 2   DEPOSIT_AMT       116201 non-null  object
 3   BALANCE_AMT       116201 non-null  object
dtypes: object(4)
memory usage: 3.5+ MB


In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.appName("SparkByExample").getOrCreate()

In [12]:
spark

In [28]:
df = spark.read.csv("C:/Users/acer/Downloads/fc_transaction_base.csv",header=True,inferSchema=True)

In [29]:
df.show(5)

+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+
| tran_date|     account_number|branch|product|lcy_amount|transaction_code|        description1|dc_indicator|is_salary|
+----------+-------------------+------+-------+----------+----------------+--------------------+------------+---------+
|09/06/2020|02XYZXYZ10017529992|    15|    SBA|   14500.0|              CI|ATM WDL /00463701...|    withdraw|        0|
|14/06/2020|02XYZXYZ10017529992|    15|    SBA|   20000.0|              CI|ATM WDL /00463701...|    withdraw|        0|
|01/06/2020|02XYZXYZ10017517823|    15|    SBA|    4000.0|              CI|ATM WDL /NICAV701...|    withdraw|        0|
|05/06/2020|02XYZXYZ10017517823|    15|    SBA|     500.0|              CI|ATM WDL /NICAV701...|    withdraw|        0|
|12/06/2020|02XYZXYZ10017517823|    15|    SBA|    5000.0|              CI|ATM WDL /NICAV701...|    withdraw|        0|
+----------+-------------------+------+-

In [30]:
df.printSchema()

root
 |-- tran_date: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- branch: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- lcy_amount: double (nullable = true)
 |-- transaction_code: string (nullable = true)
 |-- description1: string (nullable = true)
 |-- dc_indicator: string (nullable = true)
 |-- is_salary: integer (nullable = true)



In [31]:
df = df.select("account_number","lcy_amount")

In [32]:
df.show(5)

+-------------------+----------+
|     account_number|lcy_amount|
+-------------------+----------+
|02XYZXYZ10017529992|   14500.0|
|02XYZXYZ10017529992|   20000.0|
|02XYZXYZ10017517823|    4000.0|
|02XYZXYZ10017517823|     500.0|
|02XYZXYZ10017517823|    5000.0|
+-------------------+----------+
only showing top 5 rows



In [33]:
from pyspark.sql.functions import *

In [34]:
!pip install cryptography

You should consider upgrading via the 'C:\Users\acer\PycharmProjects\PySpark\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [35]:
from pyspark.sql.functions import udf, lit, col
from cryptography.fernet import Fernet

In [37]:
enc_df = df.withColumn('encrypted_account_number', expr("base64(aes_encrypt(account_number, '1234567890abcdef', 'ECB', 'PKCS'))")).select("encrypted_account_number","lcy_amount")
enc_df.show(5)

+------------------------+----------+
|encrypted_account_number|lcy_amount|
+------------------------+----------+
|    LLY2iKGyxElG5+2r0...|   14500.0|
|    LLY2iKGyxElG5+2r0...|   20000.0|
|    E97z6MmS7wv2ZB4L/...|    4000.0|
|    E97z6MmS7wv2ZB4L/...|     500.0|
|    E97z6MmS7wv2ZB4L/...|    5000.0|
+------------------------+----------+
only showing top 5 rows



In [38]:
enc_df.printSchema()

root
 |-- encrypted_account_number: string (nullable = true)
 |-- lcy_amount: double (nullable = true)



In [39]:
enc_df = enc_df.select("*").toPandas()

In [40]:
enc_df.head()

,encrypted_account_number,lcy_amount
0,LLY2iKGyxElG5+2r0eurWcPPH8xV0PpM3ad6GUa9LmI=,14500.0
1,LLY2iKGyxElG5+2r0eurWcPPH8xV0PpM3ad6GUa9LmI=,20000.0
2,E97z6MmS7wv2ZB4L/z/jqnPyoXhfIacThZ+ov5upc3M=,4000.0
3,E97z6MmS7wv2ZB4L/z/jqnPyoXhfIacThZ+ov5upc3M=,500.0
4,E97z6MmS7wv2ZB4L/z/jqnPyoXhfIacThZ+ov5upc3M=,5000.0


In [25]:
KAFKA_TOPIC_NAME_CONS = "Topic3"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'

In [41]:
if __name__ == "__main__":
    print("Kafka Producer Application Started ... ")

    kafka_producer_obj = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS_CONS,
                                       value_serializer=lambda x: x.encode('utf-8'))
    
    enc_df =enc_df.head(50)
    # enc_df['id'] = np.arange(len(enc_df)) 
    df_list = enc_df.to_dict(orient="records")
    message_list = []
    message = None
    for message in df_list:
        
        message_fields_value_list = []       
        message_fields_value_list.append(message["encrypted_account_number"])
        message_fields_value_list.append(message["lcy_amount"])
        message = ','.join(str(v) for v in message_fields_value_list)
        print("Message Type: ", type(message))
        print("Message: ", message)
        kafka_producer_obj.send(KAFKA_TOPIC_NAME_CONS, message)
        time.sleep(1)


    print("Kafka Producer Application Completed. ")

Kafka Producer Application Started ... 
Message Type:  <class 'str'>
Message:  LLY2iKGyxElG5+2r0eurWcPPH8xV0PpM3ad6GUa9LmI=,14500.0
Message Type:  <class 'str'>
Message:  LLY2iKGyxElG5+2r0eurWcPPH8xV0PpM3ad6GUa9LmI=,20000.0
Message Type:  <class 'str'>
Message:  E97z6MmS7wv2ZB4L/z/jqnPyoXhfIacThZ+ov5upc3M=,4000.0
Message Type:  <class 'str'>
Message:  E97z6MmS7wv2ZB4L/z/jqnPyoXhfIacThZ+ov5upc3M=,500.0
Message Type:  <class 'str'>
Message:  E97z6MmS7wv2ZB4L/z/jqnPyoXhfIacThZ+ov5upc3M=,5000.0
Message Type:  <class 'str'>
Message:  RaOXBO26LRxTdo9rDMiYtXzd5pTTGC6MMp9M5rpy2n0=,5000.0
Message Type:  <class 'str'>
Message:  RaOXBO26LRxTdo9rDMiYtaEEsmHRJc/qBmMDCjDDC78=,5010.0
Message Type:  <class 'str'>
Message:  IePg4bkHt0xLcjga9BDXpSVpeEg4tUdytwyhkv4NxNQ=,15000.0
Message Type:  <class 'str'>
Message:  IePg4bkHt0xLcjga9BDXpSVpeEg4tUdytwyhkv4NxNQ=,10000.0
Message Type:  <class 'str'>
Message:  IePg4bkHt0xLcjga9BDXpSVpeEg4tUdytwyhkv4NxNQ=,5000.0
Message Type:  <class 'str'>
Message:  h6X5wcaj